<a href="https://colab.research.google.com/github/piesauce/llm-playbooks/blob/ateng%2Fchatwithyourpdf/ChatwithyourPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Your First AI App: Chat with your PDF

In [1]:
# Install required dependencies using pip and apt-get
!pip install -q gradio langchain unstructured[pdf] sentence-transformers openai chromadb langchain_community tiktoken pyocr
!apt-get -qq install poppler-utils libtesseract-dev tesseract-ocr

# Import necessary libraries
import os
import traceback
from langchain_community.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import gradio as gr
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
def process_pdf(file_path):
    """Process PDF with fallback strategies"""
    # Function to process the PDF and create a vector store
    try:
        # Try loading with PyPDFLoader first
        try:
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            # If PyPDFLoader fails, use UnstructuredPDFLoader with OCR
        except:
            loader = UnstructuredPDFLoader(file_path, strategy="ocr_only")
            documents = loader.load()

        # Create embeddings using HuggingFaceEmbeddings
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        # Build a Chroma vector store from the documents and embeddings
        return Chroma.from_documents(documents, embeddings)
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None

def main():
    # Main function to run the chatbot
    # Set OpenAI API key (replace with your actual key)
    # api_key = input("Enter your OpenAI API key (sk-...): ").strip()
    os.environ["OPENAI_API_KEY"] = "PUT YOUR OPEN AI KEY HERE"

    # Upload PDF file using Google Colab's files.upload()
    print("\nUpload a PDF file:")
    uploaded = files.upload()
    # Exit if no file is uploaded
    if not uploaded:
        print("No file uploaded. Exiting.")
        return

    # Get the filename of the uploaded PDF
    file_name = next(iter(uploaded))
    # Process the PDF and create the vector store
    vector_store = process_pdf(file_name)

    # Exit if PDF processing fails
    if not vector_store:
        print("Failed to process PDF. Exiting.")
        return

    # Setup conversation chain with memory and retriever
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    qa = ConversationalRetrievalChain.from_llm(
        ChatOpenAI(temperature=0.1),
        vector_store.as_retriever(search_kwargs={"k": 3}),
        memory=memory
    )

    # Start the chat loop
    print("\nChat with your PDF (type 'exit' to quit)")
    while True:
        # Get user's query
        query = input("\nYour question: ").strip()
        # Exit if the user types 'exit' or 'quit'
        if query.lower() in ['exit', 'quit']:
            break

        # Get the chatbot's answer and print it
        try:
            result = qa({"question": query})
            print(f"\nAnswer: {result['answer']}")
        # Handle exceptions during chat
        except Exception as e:
            print(f"Error: {str(e)}")

# Run the main function if the script is executed
if __name__ == "__main__":
    main()


Upload a PDF file:


Saving AI Modalities_ Understanding the Differences _ Gamma.pdf to AI Modalities_ Understanding the Differences _ Gamma.pdf


<ipython-input-2-e0c265b3b83b>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

<ipython-input-2-e0c265b3b83b>:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
<ipython-input-2-e0c265b3b83b>:41: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(temperature=0.1),



Chat with your PDF (type 'exit' to quit)

Your question:  what is the document about? 


<ipython-input-2-e0c265b3b83b>:54: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"question": query})



Answer: I don't know, could you provide me with more information or context about the document you are referring to?

Your question: what is the PDF about? 

Answer: I don't know.

Your question: what are ai modalities? 

Answer: AI modalities are different ways in which artificial intelligence interacts with and processes information. Some examples of AI modalities include text-to-text, text-to-speech, speech-to-text, image-to-text, image-to-image, and speech-to-speech. Each modality serves distinct purposes and can be combined to create powerful multi-modal AI systems. These modalities are used in various applications such as natural language processing, machine translation, voice recognition, virtual assistants, AI art generation, and medical imaging.

Your question: what are the types of ai modalities

Answer: The types of AI modalities include Text to Text, Text to Speech, Speech to Text, Image to Text, Image to Image, and Speech to Speech. Each modality serves distinct purposes 

KeyboardInterrupt: Interrupted by user